1.topsis

In [7]:
import pandas as pd
import numpy as np

# Read Excel file
file_path = r"E:\Users\Sample Project files\Sample Project files\telemetry-main\data\Appendix 1 Data of 402 suppliers in the past five years.xlsx"
xls = pd.ExcelFile(file_path)

# Read the two data sheets
order_df = xls.parse('The company’s order volume （m³）')
supply_df = xls.parse('Supplier’s supply volume (m³)')

# Remove spaces from column names
order_df.columns = order_df.columns.str.strip()
supply_df.columns = supply_df.columns.str.strip()

# Extract the numerical part (starting from the 3rd column)
order_values = order_df.iloc[:, 2:].values
supply_values = supply_df.iloc[:, 2:].values

# Calculate key metrics
total_order = order_values.sum(axis=1)
total_supply = supply_values.sum(axis=1)
supply_rate = np.divide(total_supply, total_order, out=np.zeros_like(total_supply, dtype=float), where=total_order!=0)
supply_std = np.std(supply_values, axis=1)

# Material type weighting
material_weights = order_df["Material Classification"].map({'A': 1.2, 'B': 1.1, 'C': 1.0}).values

# Normalization
supply_volume_score = (total_supply - total_supply.min()) / (total_supply.max() - total_supply.min())
supply_stability_score = 1 / (1 + supply_std)

# Comprehensive scoring
final_score = (0.4 * supply_rate + 
               0.3 * supply_volume_score + 
               0.3 * supply_stability_score) * material_weights

# Organize the results
supplier_scores = order_df[["Supplier ID", "Material Classification"]].copy()
supplier_scores["Total Order Quantity"] = total_order
supplier_scores["Total Supply Quantity"] = total_supply
supplier_scores["Supply Completion Rate"] = supply_rate
supplier_scores["Supply Fluctuation"] = supply_std
supplier_scores["Overall Score"] = final_score

# Select the top 50
top_50_suppliers = supplier_scores.sort_values(by="Overall Score", ascending=False).head(50)

# Export to Excel file
top_50_suppliers.to_excel("Top_50_Suppliers.xlsx", index=False)